In [1]:
import pandas as pd
import emoji

# Load the training and test datasets
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')

# Drop unnecessary columns
train = train.drop('Unnamed: 0', axis=1)
test = test.drop('Unnamed: 0', axis=1)

# Create emoji mapping
mapping = pd.read_csv("Mapping.csv")
mapping = mapping.set_index('number')
mapping_dict = mapping['emoticons'].to_dict()

In [2]:
import re
from nltk.tokenize import word_tokenize

def clean_text(text):
  """
  Cleans the text by removing punctuation, converting to lowercase, and tokenizing the words.
  """
  text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
  text = text.lower()
  tokens = word_tokenize(text)
  return tokens

# Clean the text data
train['TEXT'] = train['TEXT'].apply(clean_text)
test['TEXT'] = test['TEXT'].apply(clean_text)

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Create a tokenizer
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train['TEXT'])

# Convert text to sequences
train_sequences = tokenizer.texts_to_sequences(train['TEXT'])
test_sequences = tokenizer.texts_to_sequences(test['TEXT'])

# Pad sequences to a fixed length
max_len = 100
train_sequences = pad_sequences(train_sequences, maxlen=max_len)
test_sequences = pad_sequences(test_sequences, maxlen=max_len)

# Extract emoji counts
train_emojis = train['TEXT'].apply(lambda x: sum([1 for w in x if w in mapping_dict]))
test_emojis = test['TEXT'].apply(lambda x: sum([1 for w in x if w in mapping_dict]))

# Combine features
train_features = [train_sequences, train_emojis]
test_features = [test_sequences, test_emojis]

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, SimpleRNN, Embedding
from tensorflow.keras.utils import to_categorical
import tensorflow as tf


# Build the model
model = Sequential()
model.add(Embedding(5000, 128, input_length=max_len))
model.add(LSTM(64))
model.add(Dense(4, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(train_sequences, train_emojis, epochs=10, validation_split=0.2)


Epoch 1/10


ValueError: Arguments `target` and `output` must have the same rank (ndim). Received: target.shape=(32,), output.shape=(32, 4)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Create a tokenizer
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train['TEXT'])

# Convert text to sequences
train_sequences = tokenizer.texts_to_sequences(train['TEXT'])
test_sequences = tokenizer.texts_to_sequences(test['TEXT'])

# Pad sequences to a fixed length
max_len = 100
train_sequences = pad_sequences(train_sequences, maxlen=max_len)
test_sequences = pad_sequences(test_sequences, maxlen=max_len)

# Extract emoji counts
train_emojis = train['TEXT'].apply(lambda x: sum([1 for w in x if w in mapping_dict]))
test_emojis = test['TEXT'].apply(lambda x: sum([1 for w in x if w in mapping_dict]))

# Combine features
train_features = [train_sequences, train_emojis]
test_features = [test_sequences, test_emojis]

In [ ]:
# Evaluate the model
_, accuracy = model.evaluate(test_features, to_categorical(test['Label']))
print('Accuracy: {}'.format(accuracy))